<a href="https://colab.research.google.com/github/Brotherswords/DeepLearning_Project/blob/main/CSCI4931_WUN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Brian H., Jake M., Lavan V., Matthew W.
# CSCI 4931 - Deep Learning Final Project
# Wave-U-Net ح

In [ ]:
# Environment
!pip install librosa tensorflow
!pip install tensorflow

In [ ]:
!pip install tensorflow

In [ ]:
# Wave-U-Net Model
import tensorflow as tf
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling1D
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Concatenate, Normalization, BatchNormalization

In [ ]:
print(tf.__version__)

2.14.0


In [ ]:
# mount Google Drive to the Colab VM
from google.colab import drive
import os
import zipfile

drive.mount('/content/drive')

# Mount Google Drive
drive.mount('/content/drive')

# Replace 'YourFolderName' with the name of your folder
folder_path = '/content/drive/My Drive/datasets/DSD100.zip'

# # List all files in the directory -> use this to figure out the exact
# for file in os.listdir(folder_path):
#     print(file)
print("Mounted Successfully")


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted Successfully


In [ ]:
import zipfile

# Path to the zip file
# Its not Gojover yet
# Path to the DSD100 zip file
zip_file_path = '/content/drive/My Drive/datasets/DSD100.zip'

# Function to process the zip file
mixtures_dict = {}
sources_dict = {}

# Function to process the zip file
def process_dsd100_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if 'Mixtures/Test/' in file and file.endswith('mixture.wav'):
                key = file.split('/')[-2]  # Extract song key
                # print(key)
                mixtures_dict[key] = file
            elif 'Sources/Test/' in file and file.endswith(('.wav')):
                key = file.split('/')[-2]  # Extract song key
                if key not in sources_dict:
                    sources_dict[key] = []
                sources_dict[key].append(file)


# Process the DSD100 zip file
process_dsd100_zip(zip_file_path)

# Create Mixtures and Sources lists
Mixtures = []
Sources = []

for key, mixture_path in mixtures_dict.items():
    Mixtures.append(mixture_path)
    Sources.append(sources_dict.get(key, []))  # Get the sources, or an empty list if not found

# Displaying the first few paths for verification
print("First mixture:", Mixtures[0])
print("First source tuple:", Sources[0])


# # Open the zip file and list its contents
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     for file in zip_ref.namelist():
#         print(file)

First mixture: DSD100/Mixtures/Test/050 - Zeno - Signs/mixture.wav
First source tuple: ['DSD100/Sources/Test/050 - Zeno - Signs/vocals.wav', 'DSD100/Sources/Test/050 - Zeno - Signs/other.wav', 'DSD100/Sources/Test/050 - Zeno - Signs/drums.wav', 'DSD100/Sources/Test/050 - Zeno - Signs/bass.wav']


In [ ]:
#Play the music to show that it is working
from IPython.display import Audio
def extract_and_play(file_path_in_zip):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        if file_path_in_zip in zip_ref.namelist():
            # Create a complete path for extraction
            filename = os.path.basename(file_path_in_zip)
            temp_path = '/tmp/' + filename
            # Extract the file
            zip_ref.extract(file_path_in_zip, '/tmp/')
            extracted_file_path = os.path.join('/tmp/', file_path_in_zip)
            print(f"Extracted to: {extracted_file_path}")

            # Check if the file exists at the extracted path
            if os.path.exists(extracted_file_path):
                print(extracted_file_path)
                return Audio(filename=extracted_file_path)
            else:
                print("Extracted file not found.")
                return None
        else:
            print("File not found in the zip archive.")
            return None

if Mixtures:
    audio_widget = extract_and_play(Mixtures[0])
    if audio_widget is not None:
        print("Valid audio widget")
        display(audio_widget)
    else:
        print("Audio_widget is none")
else:
    print("Mixtures list is empty.")

Extracted to: /tmp/DSD100/Mixtures/Test/050 - Zeno - Signs/mixture.wav
/tmp/DSD100/Mixtures/Test/050 - Zeno - Signs/mixture.wav
Valid audio widget


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers.normalization import batch_normalization
from tensorflow.python.ops.gen_nn_ops import BatchNormWithGlobalNormalizationGrad
# Wave-U-Net: 1D Audio Processing
# Based on U-Net, using downsampling and upsampling blocks

def BasicWaveUNet(input_shape=(None, 1), num_filters=[16, 32, 64, 128, 256, 512], filter_length=15):

    #input shape parameters
    batch_size = None
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Downsample Layers
    skips = []
    x = inputs
    for filters in num_filters:
        x = Conv1D(filters, filter_length, activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        skips.append(x)
        x = MaxPooling1D(2)(x)

    # Bottleneck Layer
    x = Conv1D(num_filters[-1], filter_length, activation='relu', padding='same')(x)

    # Upsample Layer
    for filters in reversed(num_filters[:-1]):
        x = UpSampling1D(2)(x)
        skip = skips.pop()
        x = Concatenate()([x, skip])
        x = Conv1D(filters, filter_length, activation='relu', padding='same')(x)

    # Bottleneck Layer
    x = Conv1D(num_filters[-1], filter_length, activation='relu', padding='same')(x)

    # Upsample Layers
    for filters in reversed(num_filters[:-1]):
        x = UpSampling1D(2)(x)
        skip = skips.pop()
        x = Concatenate()([x, skip])
        x = Conv1D(filters, filter_length, activation='relu', padding='same')(x)

    # Output Layer
    # Updating the number of filters to 4, one for each output channel (vocals, drums, bass, other)
    outputs = Conv1D(4, filter_length, activation='linear', padding='same')(x)

    # Create the Model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

# Create Wave-U-Net model
basic_wave_model = BasicWaveUNet()
basic_wave_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, None, 16)     256         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, None, 16)    64          ['conv1d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 dropout (Dropout)              (None, None, 16)     0           ['batch_normalization_1[0][0]

Worth an investigation possibly: https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv1D